# Package loading and basic configurations

In [1]:
%load_ext autoreload
%autoreload 2

# load dependencies'
import pandas as pd
import geopandas as gpd

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig, Visualiser, Preprocessing, TrackConverter

# create an initial but optional config and an api client
config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

ImportError: cannot import name 'Visualiser' from 'envirocar' (/home/imke/Desktop/CITRAM/enviroCar_fork/envirocar_preprocessing/lib/python3.7/site-packages/envirocar/__init__.py)

# Querying enviroCar Tracks

The following cell queries tracks from the enviroCar API. It defines a bbox for the area of Gievenbeck, Münster (Germany) and requests 50 tracks. The result is a GeoDataFrame, which is a geo-extended Pandas dataframe from the GeoPandas library. It contains all information of the track in a flat dataframe format including a specific geometry column. 

In [3]:
bbox =BboxSelector([
    7.004130554199218, # min_x
    52.50590322041212, # min_y
    7.990351104736328, # max_x
    52.99874790276371  # max_y
])


# issue a query
track_df = track_api.get_tracks(bbox=bbox, num_results=100) # requesting 10 tracks inside the bbox
# track_df

In [4]:
track_df

,id,time,geometry,GPS Accuracy.value,GPS Accuracy.unit,Intake Pressure.value,Intake Pressure.unit,GPS Altitude.value,GPS Altitude.unit,GPS VDOP.value,...,O2 Lambda Voltage ER.value,O2 Lambda Voltage ER.unit,O2 Lambda Voltage.value,O2 Lambda Voltage.unit,track.appVersion,track.touVersion,Short-Term Fuel Trim 1.value,Short-Term Fuel Trim 1.unit,Long-Term Fuel Trim 1.value,Long-Term Fuel Trim 1.unit
0,5e08bc845bc8db428964203f,2019-12-29T12:11:19,POINT (7.27089 53.52839),1.899800,%,36.000001,kPa,45.999999,m,0.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e08bc845bc8db4289642041,2019-12-29T12:11:25,POINT (7.27071 53.52830),2.033838,%,56.470588,kPa,44.999999,m,0.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e08bc845bc8db4289642042,2019-12-29T12:11:30,POINT (7.27037 53.52828),1.500000,%,35.000000,kPa,44.000000,m,0.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e08bc845bc8db4289642043,2019-12-29T12:11:35,POINT (7.27023 53.52824),2.000000,%,38.000001,kPa,44.999999,m,0.610282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e08bc845bc8db4289642044,2019-12-29T12:11:40,POINT (7.27023 53.52824),2.500000,%,41.190981,kPa,45.000001,m,0.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,51efeb89e4b058cd3d5b1596,2013-07-24T15:01:02,POINT (7.19301 53.63090),NaN,NaN,107.000000,kPa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,51efeb89e4b058cd3d5b1598,2013-07-24T15:01:08,POINT (7.19308 53.63090),NaN,NaN,107.000000,kPa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,51efeb89e4b058cd3d5b159a,2013-07-24T15:01:14,POINT (7.19313 53.63100),NaN,NaN,107.000000,kPa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,51efeb89e4b058cd3d5b159c,2013-07-24T15:01:21,POINT (7.19313 53.63100),NaN,NaN,106.000000,kPa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Aggregation

### Spatio-temporal aggregation based on weekday and hour

In [5]:
#Spatio temporally aggregate CO2 mean value with 1000 m of grid side
grid,subGrid=Visualiser.spatioTemporalAggregation(track_df,"Speed.value","mean",1000)

In [ ]:
#Visualize spatio-temporal aggregate
Visualiser.MosaicPlot(grid,subGrid,'Speed.value_mean')

### Grid wise Spatial Aggregation

In [ ]:
summary=Visualiser.aggregateByGrid(track_df,"Speed.value","mean",1000)

In [ ]:
choropleth=Visualiser.plotAggregate(summary,"mean_Speed.value")
choropleth

### Temporal Aggregation on hourly basis

In [ ]:
#Aggregate data temporally over the whole dataset
import seaborn as sns
x=Visualiser.aggregateHourly(track_df,'Speed.value','mean')
dayhourPivot=x.pivot(index='hour_',columns='weekday_',values='Speed.value_mean')
dayhourPivot
axs=sns.heatmap(dayhourPivot)

### Cluster  origin of tracks using dbscan algorithm

In [ ]:
#Get origin and destination of tracks
origin,destination=Visualiser.OriginAndDestination(track_df)

In [ ]:
#Get cluster centers
clusterCentres=Visualiser.getClusters(origin,3,2)

### Show clusters of track origins with bubblemap

In [ ]:
#Show cluster for origin of tracks overlaid with track points
clus=Visualiser.showClusters(clusterCentres,track_df)
clus